# Lab 10
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marts-dev/geog510-test/blob/main/lab10.ipynb)

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

## Exercise 1: Visualizing DEM Data
Find a DEM dataset in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) and clip it to a specific area (e.g., your country, state, or city). Display it with an appropriate color palette. For example, the sample map below shows the DEM of the state of Colorado.

### If you want to clip before adding the layer

In [16]:
# This represents the Area of Interest
aoi = ee.Geometry.Rectangle([116.47314, 4.747796, 128.060339, 18.985783])

m = geemap.Map()
dataset = ee.Image('NASA/NASADEM_HGT/001').clip(aoi)
m.centerObject(aoi, zoom=6)
elevation = dataset.select('elevation')
vis_params = {'min': 0, 'max': 2500, 'palette': 'terrain'}
# Set elevation <= 0 as transparent then add it to the map.
m.add_layer(elevation.updateMask(elevation.gt(0)), vis_params, 'Elevation')

m

Map(center=[11.861980446390639, 122.26673950000003], controls=(WidgetControl(options=['position', 'transparent…

## If you would like to create a clip based on rectangle drawn on the map
Using the drawing tool in the left side, create the bounds you would like to focus on.
After that, execute the code below.

In [4]:
if m.user_roi is not None:
  image = dataset.clip(m.user_roi).select('elevation')
  m.centerObject(m.user_roi, zoom=8)
  m.layers[1].visible = False
  m.add_layer(image.updateMask(image.gt(0)), vis_params, "Clipped DEM")

## Exercise 2: Cloud-Free Composite with Sentinel-2 or Landsat

In [5]:
# This represents the Area of Interest
aoi = ee.Geometry.Rectangle([116.47314, 4.747796, 128.060339, 18.985783])

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
  ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(aoi)
  .filterDate('2023-01-01', '2025-01-30')
  # Pre-filter to get less cloudy granules.
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  .map(mask_s2_clouds)
)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}
m = geemap.Map()
m.centerObject(aoi, zoom=5)
m.add_layer(dataset.median(), visualization, 'RGB')
m

Map(center=[11.861980446390639, 122.26673950000003], controls=(WidgetControl(options=['position', 'transparent…

## Exercise 3: Visualizing NAIP Imagery
Use [NAIP imagery](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) to create a cloud-free imagery for a U.S. county of your choice. For example, the sample map below shows a cloud-free true-color composite of NAIP imagery for Knox County, Tennessee. Keep in mind that there might be some counties with the same name in different states, so make sure to select the correct county for the selected state.

In [6]:
counties = ee.FeatureCollection('TIGER/2018/Counties')
county = counties.filter(ee.Filter.eq('NAME', 'Travis')).filter(ee.Filter.eq('STATEFP', '48'))

dataset = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(county)
    .filterDate('2018-01-01', '2018-12-31')
)

fcir = dataset.select(['N', 'R', 'G'])
vis_options = {
  'min': 0.0,
  'max': 255.0,
  'color': 'red',
  'fillColor': '00000000',
  'width': 2,
}
## Clip NAIP imagery to the county boundary
masked = fcir.mosaic().updateMask(ee.Image.constant(1).clip(county))
## Combine the masked imagery with the county outline
fcir_blend = masked.blend(county.style(
    color='red', fillColor='00000000', width=1
))
m = geemap.Map()
m.centerObject(county, zoom=10)
#m.addLayer(masked, vis_options, 'NAIP')
#m.addLayer(county.style(color='red', fillColor='00000000',width=2), {}, 'County Outline')
m.add_layer(fcir_blend, {}, 'NAIP with County Outline')
m

Map(center=[30.334674542028303, -97.781806607209], controls=(WidgetControl(options=['position', 'transparent_b…

## Exercise 4: Visualizing Watershed Boundaries
Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [7]:
dataset =ee.FeatureCollection('USGS/WBD/2017/HUC04')

In [8]:
m = geemap.Map(center=[40.93, -98.17], zoom=4)
m.add_layer(dataset.style(color='blue', fillColor='00000000', width=2), {}, 'Watershed Boundaries')
m

Map(center=[40.93, -98.17], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

## Exercise 5: Visualizing Land Cover Change
Use the USGS National Land Cover Database and US Census States to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [9]:
from geemap.legends import builtin_legends
states = ee.FeatureCollection('TIGER/2018/States')
state = states.filter(ee.Filter.eq('NAME', 'Massachusetts'))

m = geemap.Map()
m.centerObject(state, zoom=8)

In [10]:
nlcd_2001_landcover = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2001').select('landcover')
nlcd_2019_landcover = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019').select('landcover')

In [11]:
left_layer = geemap.ee_tile_layer(
  nlcd_2001_landcover.updateMask(ee.Image.constant(1).clip(state)),
  {},
  name='NLCD 2001',
  shown=True,
)
right_layer = geemap.ee_tile_layer(
  nlcd_2019_landcover.updateMask(ee.Image.constant(1).clip(state)),
  {},
  name='NLCD 2019',
  shown=True,
)

m.split_map(left_layer, right_layer)
m.add_legend(builtin_legend="NLCD", max_width="100px", height="455px", title='NLCD Land Cover Type')
m

Map(center=[42.161779492134215, -71.49747297364661], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Exercise 6: Landsat Timelapse Animation
Generate a timelapse animation using Landsat data to show changes over time for a selected region.

In [12]:
m = geemap.Map()
roi = ee.Geometry.Rectangle(120.885897, 14.492245, 121.038244, 14.631476)
m.add_layer(roi)
m.centerObject(roi, zoom=12)

In [20]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="manila.gif",
    start_year=1990,
    end_year=2025,
    bands=["SWIR1", "NIR", "Blue"],
    frames_per_second=2,
    title="Manila",
    font_color="red",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: f:\Github\geog510-test\manila.gif


Output()